# Homework

Generated tasks from task_01:  

| parameter | value |
| --------- | :-----: |
| Type | Regression |
| Data set | Boston Housing Data Set |
| Method 1 | Linear regression |
| Method 2 | Nadaraya-Watson |

python 3.8.5 64-bit (conda)

## Imports

In [1]:
# Data imports
from sklearn.datasets import load_boston

# Data normalization
from sklearn.preprocessing import MinMaxScaler

# Used backend libraries
from scipy.spatial.distance import cdist
import numpy as np
import math

# For tests and representation of results
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd


## Data loading

In [2]:
# Load dataset
boston_dataset = load_boston()
# Variables for learning
X_raw = boston_dataset.data
Y_raw = boston_dataset.target
# Show data
print(f"X shape: {X_raw.shape}")
print(f"Y shape: {Y_raw.shape}")
pd.DataFrame(X_raw, columns=boston_dataset.feature_names)

X shape: (506, 13)
Y shape: (506,)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


## Data preprocessing

Is used minmax normalization for data:

$$
x_{norm} = \frac{x - \displaystyle\min_{x \in X}(x)}{\displaystyle\max_{x \in X}(x) - \displaystyle\min_{x \in X}(x)}
$$


In [3]:
# Normalization
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X_raw)
# Show results
pd.DataFrame(X_norm, columns=boston_dataset.feature_names)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.000633,0.00,0.420455,0.0,0.386831,0.580954,0.681771,0.122671,0.000000,0.164122,0.893617,0.987619,0.219095
502,0.000438,0.00,0.420455,0.0,0.386831,0.490324,0.760041,0.105293,0.000000,0.164122,0.893617,1.000000,0.202815
503,0.000612,0.00,0.420455,0.0,0.386831,0.654340,0.907312,0.094381,0.000000,0.164122,0.893617,1.000000,0.107892
504,0.001161,0.00,0.420455,0.0,0.386831,0.619467,0.889804,0.114514,0.000000,0.164122,0.893617,0.991301,0.131071


## Linear Regression

In [4]:
class LinearRegression(object):

    def __init__(self, rate: float = 0.2, steps_count: int = 100):
        self.rate_ = rate
        self.n_steps_ = steps_count
        self.weights_ = None
    
    @staticmethod
    def gradient(X, Y, weights):
        """
        Compute gradient for linear regression.

        Parameters
        ----------
        X: array_like
            Input dataset of shape (m, n + 1)
        Y: array_like
            Array of real values of shape (m, )
        weights: array_like
            Weights of shape (n + 1, ) for linear regression learning

        Returns
        -------
        grad_cost: array_like
            Computed gradient for input weights and data
        """
        return (2 / X.shape[0]) * np.dot(X.T, (np.dot(X, weights) - Y))

    def fit(self, X, Y):
        """
        Fit linear model.

        Parameters
        ----------
        X: array_like
            Input dataset of shape (m, n)
        Y: array_like
            Array of real values of shape (m, )
        
        Returns
        -------
        None
        """

        # Add bias term for input data
        X_train = np.hstack((np.ones((X.shape[0], 1)), X))
        # Started weights initialization 
        self.weights_ = np.zeros(X_train.shape[1])

        # Learning loop
        for _ in range(self.n_steps_):
            self.weights_ = self.weights_ - self.rate_ * LinearRegression.gradient(X_train, Y, self.weights_)
        
    def predict(self, x):
        """
        Predict using the linear regression.

        Parameters
        ----------
        x: array_like
            Samples of shape (samples_count, m)

        Returns
        -------
        pred_values: array_like
            Predicted values of shape (samples_count, )
        """
        return np.dot(np.hstack((np.ones((x.shape[0], 1)), x)), self.weights_)

## Nadaraya-Watson method

$$
K(d) - kernel,\ h(x, X) - Parzen\ window\ width\ (static\ or\ dynamic) \\
a(x, X^l)= \frac{\sum_{i=1}^{l} y_i K(\frac{\rho(x, x_i)}{h(x, X)})}{\sum_{i=1}^{l} K(\frac{\rho(x, x_i)}{h(x, X)})}
$$

### Kernels
#### Epanechnikov kernel
$$
E(d) = \frac{3}{4} (1 - d^2)\cdot\mathbb{1}\{|d| \leq 1\}
$$
####  Gaussian kernel
$$
G(d) = \frac{1}{\sqrt{2 \pi}} e^{\frac{-d^2}{2}}
$$
#### Tophat (Uniform) kernel
$$
P(d) = \frac{1}{2}\cdot\mathbb{1}\{|d| \leq 1\}
$$
#### Triangular kernel
$$
T(d) = (1 - |d|)\cdot\mathbb{1}\{|d| \leq 1\}
$$
#### Quartic kernel
$$
Q(d) = \frac{15}{16}(1 - d^2)^2\cdot\mathbb{1}\{|d| \leq 1\}
$$
#### Sigmoid kernel
$$
S(d) = \frac{2}{\pi}\frac{1}{e^d + e^{-d}}
$$

In [5]:
# Epanechnikov kernel
def E(distance, window_width):
    dist = np.array(distance) / window_width
    return 0.75 * (1 - dist ** 2) * (np.abs(dist) <= 1)

# Gaussian kernel
def G(distance, window_width):
    dist = np.array(distance) / window_width
    return (1 / math.sqrt(2 * math.pi)) * np.exp((-(dist ** 2)) / 2)

# Tophat (Uniform) kernel
def P(distance, window_width):
    dist = np.array(distance) / window_width
    return 0.5 * (np.abs(dist) <= 1)

# Triangular kernel
def T(distance, window_width):
    dist = np.array(distance) / window_width
    return (1 - np.abs(dist)) * (np.abs(dist) <= 1)

# Quartic kernel
def Q(distance, window_width):
    dist = np.array(distance) / window_width
    return (15.0 / 16.0) * ((1 - dist ** 2) ** 2) * (np.abs(dist) <= 1)

# Sigmoid kerne
def S(distance, window_width):
    dist = np.array(distance) / window_width
    return (2 / math.pi) / (np.exp(dist) + np.exp(-dist))

In [6]:
class NadarayaWatson(object):

    def __init__(self, kernel=E, k: int = 3):
        self.X_ = None
        self.Y_ = None
        self.kernel_ = kernel
        self.parzen_window_ = None
        self.k_ = k

    def fit(self, X, Y):
        """
        Fit Nadaraya-Watson model.

        Parameters
        ----------
        X: array_like
            Input dataset of shape (m, n)
        Y: array_like
            Array of real values of shape (m, )
        
        Returns
        -------
        None
        """
        # For Nadaraya-Watson simple is saved samples
        self.X_ = np.array(X)
        self.Y_ = np.array(Y)
        
    def predict(self, x):
        """
        Predict using the Nadaraya-Watson method.

        Parameters
        ----------
        x: array_like
            Samples of shape (samples_count, m)

        Returns
        -------
        pred_values: array_like
            Predicted values of shape (samples_count, )
        """
        distances = cdist(x, self.X_)
        # Use dynamic parzen window (k+1 neighbor)
        h = np.sort(distances)[:,self.k_]
        kernel_res = self.kernel_(distances, h[:, None])
        features_sum = np.sum(self.Y_ * kernel_res, axis=-1)
        # Add small constant to total sum to avoid float arithmetic warning
        return features_sum / (np.sum(kernel_res, axis=-1) + 1e-10)

## Tests and Results

In [7]:
num_kfold = 10
kf = KFold(num_kfold)

def test(Model, parameters_1, parameters_2, X, Y):
    results = []
    for parameter_1 in parameters_1:
        results.append([])
        for parameter_2 in parameters_2:
            rmse_sum = 0.0
            # Split on test and train samples
            for train_indices, test_indices in kf.split(X):
                model = Model(parameter_1, parameter_2)
                model.fit(X[train_indices], Y[train_indices])
                Y_pred = model.predict(X[test_indices])
                # Calculate rmse
                rmse_sum += mean_squared_error(Y[test_indices], Y_pred, squared=False)
            # Add avarage rmse to results
            results[-1].append(rmse_sum / num_kfold)
    return results

In [8]:
# Learning rate and iterations count for linear regression
alphas, steps = [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.1, 0.2], [20, 40, 60, 80, 100, 200, 400, 800, 1500, 2000, 3000]
# Test linear regression
results = test(LinearRegression, alphas, steps, X_norm, Y_raw)
# Show results
pd.DataFrame(results, index=alphas, columns=steps)

,20,40,60,80,100,200,400,800,1500,2000,3000
0.001,20.778687,18.417748,16.453137,14.844076,13.558158,10.474570,9.462069,8.610280,7.638435,7.256743,6.845403
0.005,13.473830,10.441850,9.748350,9.463677,9.235450,8.260907,7.255757,6.615934,6.082746,5.799144,5.405022
0.010,10.402002,9.465960,9.017748,8.608273,8.258991,7.254525,6.615823,6.020351,5.404906,5.163860,4.917551
0.020,9.471408,8.605775,7.967131,7.540426,7.252063,6.615603,6.020136,5.346741,4.917459,4.816111,4.762238
0.030,9.020984,7.962974,7.379769,7.049144,6.842963,6.287378,5.616942,5.037903,4.790733,4.762228,4.787130
0.040,8.599907,7.533654,7.047483,6.788878,6.615176,6.019703,5.346301,4.889386,4.762217,4.773119,4.840584
0.050,8.242432,7.244693,6.841317,6.614969,6.440773,5.797988,5.163160,4.815971,4.767667,4.803799,4.895988
0.070,7.713019,6.898579,6.577334,6.346587,6.146858,5.467390,4.950216,4.765076,4.812734,4.877968,4.988251
0.100,7.232476,6.613998,6.286552,6.018389,5.796694,5.162284,4.815737,4.773129,4.896044,4.974975,5.075905
0.200,6.612375,6.016148,5.612526,5.342773,5.160531,4.815270,4.773145,4.913540,5.075985,5.126820,5.165297


In [9]:
# Kernels and neighbors count (dynamic parzen window usage) for Nadaraya-Watson method
kernels, neighbors = [E, G, P, T, Q, S], [1, 2, 3, 4, 5, 8, 10, 12, 15, 17, 20, 25]
# Test Nadaraya-Watson method
results = test(NadarayaWatson, kernels, neighbors, X_norm, Y_raw)
# Show results
pd.DataFrame(results, index=[k.__name__ for k in kernels], columns=neighbors)

,1,2,3,4,5,8,10,12,15,17,20,25
E,6.475398,6.082422,5.721199,5.636515,5.620032,5.563837,5.613480,5.675565,5.726785,5.741245,5.771925,5.845559
G,6.679708,6.812917,6.990575,7.063891,7.111383,7.253345,7.306131,7.348097,7.414750,7.441102,7.476916,7.527136
P,5.818329,5.621769,5.639056,5.631617,5.711742,5.865409,5.908960,5.923147,5.915575,5.953784,6.005255,6.013253
T,6.475398,6.084789,5.730255,5.635753,5.614309,5.535383,5.574117,5.628422,5.673372,5.686308,5.717320,5.788801
Q,6.475428,6.193432,5.962292,5.841294,5.796370,5.624368,5.597792,5.586039,5.589468,5.583391,5.605161,5.657234
S,7.260799,7.350942,7.512244,7.557822,7.589478,7.667073,7.696137,7.721997,7.762200,7.777821,7.802788,7.848259


Further in Russian :)

Как я и ожидал при увеличении количества итераций точность вычисления для линейной регрессии улучшается (уменьшается RMSE). Повышенные значения в конце (на 3000 итерациях) означают достижения локального минимума, а значит на дальнейших итерациях алгоритм будет "прыгать" в окрестности локального минимума. Наилучшие показатели достигаются при значениях коэффициента скорости обучения 0.03, 0.04, 0.05. При бОльших скоростях обучения мы быстрее достигаем локального минимума (что логично), но тем самым алгоритму сложнее его достигнут (собственно из большого коэффициента скорости обучения).  

Метод Надаря-Ватсона показал чуть похуже результаты (чего я не ожидал), но близкие к параметрам линейной регресии для скорости обучения 0.001, 0.002, 0.2. Однако я ожидал, что изменения ядер не будет сильно влиять на результаты регрессии (были использованы как инфинитные, так и не инфинитные ядра). Здесь стоит отметить, что инфинитные ядра оказываются хуже. Предполагаю, что это связано с тем, что "хвосты" таких ядер оказывают более существенное влияние на этом наборе данных, чем ядра, ограниченные носителем в виде индикатора.  

Мне интересно стало понять почему казалось бы простая линейная регрессия (хотя для данного учебного набора это нормальная ситуация) показывает хоть немного, но лучшие результаты. Для начала стоит отменить, что методы, основанные на ядрах чувствительны к данным. В таблице с результатами метода Надарая-Ватсона мы можем заметить, что начиная с некоторого большого количества соседей результаты ухудшаются (при большем количестве соседей мы увеличиваем размер парзеновского окна, таким образом увеличивая влияние большего числа объектов), как и на семинаре (07) мы видели небольшое переобучение на большом количевстве соседей.  

Поиследовав научную литературу, я нашёл подтверждение своим словам (например, [тут](http://www.ccas.ru/voron/download/Regression.pdf)):  
1. Метод Надарая-Ватсона чувстителем к выбросам в данных.  
2. При больших (и не только) размернотях (у нас размерност вектора данных = 13) возникают краевые эффекты, свзязаные со смещением аппроксимирующей функции относительно истинной зависмости из-за расположения объектов выборки не вокруг заданного объекта, а по одну сторону от него.

В добавок ко всему выше сказанному скажу, что исходный датасет не является большим (всего 506 векторов), что, возможно, также может делать его приятным для линейной регресии.

## Заключение

В данной работе реализованы методы линейной регрессии и регрессии методом Надаря-Ватсона. Было произведено тестирование этих методов на наборе Boston Housing Data Set и оценены результаты работы данных алгоритмов. Заключения содержат как собственную оценку результатов, так и изучение научной литературы. Процесс реализации данных алгоритмов и их непосредственное тестирование помогли лучше разобраться во внутреннем строением и особенностями предложенных методов.

Спасибо за домашнее задание :)

